<a href="https://colab.research.google.com/github/CamiloVga/Curso-IA-Aplicada/blob/main/Script_Clase_24_Diffusers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🎨 Inteligencia Artificial Aplicada para la Economía
## Universidad de los Andes

### 👨‍🏫 Profesores
- **Profesor Magistral:** [Camilo Vega Barbosa](https://www.linkedin.com/in/camilovegabarbosa/)
- **Asistente de Docencia:** [Sergio Julian Zona Moreno](https://www.linkedin.com/in/sergiozonamoreno/)

### 📚 Implementación de Stable Diffusion XL para Generación de Imágenes
Este script implementa un generador de imágenes basado en el modelo Stable Diffusion XL:

1. **Configuración y Optimización del Modelo Diffusion 🚀**

2. **Generación de Imágenes con Arquitectura Diffusion 🖼️**

3. **Interfaz Gráfica con Gradio 🌈**



In [ ]:
# Instalar bibliotecas necesarias
!pip install diffusers transformers accelerate torch gradio -q


# Importar bibliotecas
import torch
from diffusers import StableDiffusionXLPipeline
import gradio as gr

# Configurar dispositivo
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Usando dispositivo: {device}")

# Cargar modelo Stable Diffusion XL optimizado para memoria limitada
pipe = StableDiffusionXLPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype=torch.float16,            # Usar precisión de 16 bits para ahorrar memoria
    variant="fp16",                       # Usar variante de precisión media
    use_safetensors=True                  # Usar formato safetensors
)

# Optimizar para GPU
pipe = pipe.to(device)


# Función para generar imágenes
def generate_image(
    prompt,
    negative_prompt="low quality, blurry, distorted, deformed",
    num_steps=30,
    guidance_scale=7.5,
    seed=42
):
    # Configurar generador para reproducibilidad
    if seed != -1:
        generator = torch.Generator(device=device).manual_seed(seed)
    else:
        generator = None

    # Generar imagen
    image = pipe(
        prompt=prompt,
        negative_prompt=negative_prompt,
        num_inference_steps=num_steps,
        guidance_scale=guidance_scale,
        generator=generator
    ).images[0]

    return image



In [ ]:
# Crear la interfaz de Gradio
with gr.Blocks(title="Generador de Imágenes con Stable Diffusion XL") as demo:
    gr.Markdown("""
    # 🎨 Generador de Imágenes con Stable Diffusion XL
    Ingresa un prompt descriptivo para generar una imagen usando IA
    """)

    with gr.Row():
        with gr.Column():
            # Controles de entrada
            prompt = gr.Textbox(label="Prompt", placeholder="Describe la imagen que quieres generar...")
            negative_prompt = gr.Textbox(
                label="Prompt Negativo",
                value="low quality, blurry, distorted, deformed",
                placeholder="Elementos que NO quieres en la imagen"
            )

            with gr.Row():
                steps = gr.Slider(minimum=10, maximum=50, value=30, step=1, label="Pasos de Inferencia")
                guidance = gr.Slider(minimum=1.0, maximum=15.0, value=7.5, step=0.1, label="Guidance Scale")

            seed = gr.Slider(minimum=-1, maximum=2147483647, value=42, step=1, label="Seed (-1 para aleatorio)")

            generate_btn = gr.Button("Generar Imagen", variant="primary")

        # Salida: la imagen generada
        image_output = gr.Image(label="Imagen Generada")

    # Ejemplos de prompts creativos
    examples = gr.Examples(
        examples=[
            ["Un paisaje futurista con rascacielos flotantes y vehículos voladores, estilo realista"],
            ["Retrato de un economista del futuro, estilo digital art, detallado"],
            ["Visualización de datos económicos en un entorno 3D abstracto, colores vibrantes"],
            ["Una ciudad inteligente con tecnología integrada en la arquitectura, vista aérea, atardecer"]
        ],
        inputs=prompt
    )

    # Configurar la acción del botón
    generate_btn.click(
        fn=generate_image,
        inputs=[prompt, negative_prompt, steps, guidance, seed],
        outputs=image_output
    )

# Lanzar la app
demo.launch(share=True)  # share=True crea un enlace público